In [1]:
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import sys
pkg_path = "/nfs/homedirs/yuny/project-1/audio_classification"
if pkg_path not in sys.path:
    sys.path.append(pkg_path)

pkg_path = "/nfs/homedirs/yuny/project-1/foolbox"
if pkg_path not in sys.path:
    sys.path.append(pkg_path)

import os
import yaml
import torch
# from audio_classification.tools import attack_model, attack_model_for_randomize_smoothing

torch.cuda.empty_cache()
device = torch.device('cpu')
# device = (torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu'))
project_dir = '/nfs/homedirs/yuny/project-1/'
save_folder = '/nfs/homedirs/yuny/project-1/attack_results/'

In [2]:
from typing import Union, Any, Optional, Callable, Tuple
from abc import ABC, abstractmethod
import eagerpy as ep

from foolbox.devutils import flatten
from foolbox.devutils import atleast_kd
from foolbox.types import Bounds
from foolbox.models.base import Model
from foolbox.criteria import Misclassification, TargetedMisclassification
from foolbox.distances import l1, l2, linf
from foolbox.attacks.base import FixedEpsilonAttack
from foolbox.attacks.base import T
from foolbox.attacks.base import get_criterion
from foolbox.attacks.base import raise_if_kwargs

import os
import time
import types
import yaml
import torch
from audio_classification.tools import do_train, get_dataloader, get_model, get_transform
from audio_classification.tools.train_net import collate
from audio_classification.model import lit_m11, LitCRNN, SmoothClassifier
from audio_classification.data import BMWDataset, UrbanSoundDataset
from foolbox import PyTorchModel, accuracy, samples
from foolbox.attacks import LinfPGD, L2PGD, FGM, FGSM
from torch.utils.data import DataLoader
import numpy as np
import matplotlib.pyplot as plt
import torchaudio
import IPython.display as ipd
from scipy.io.wavfile import write
from audio_classification.tools.foolbox_attack import _get_loss_fn, _value_and_grad, _run

In [3]:
title = 'bmw_audio_rs_0.1'
config_path = 'logs/bmw/default/crnn-bmw-audi-rs-0.1/hparams.yaml'
pretrained_path = 'weights/bmw/crnn-bmw-audi-rs-0.1-epoch=112-val_acc=0.902.ckpt'
# attack_model_randomized_smoothing(project_dir, config_path, pretrained_path, title)

In [4]:
project="BMW"
with open(os.path.join(project_dir, config_path), "r") as config_file:
    configs = yaml.load(config_file)
configs["ATTACK"]=True
print(configs)

# use test/validattion set
if project=="BMW":
    val_set = BMWDataset(configs, [11], transform=get_transform(configs)) # actually the test set
elif project=="UrbanSound8k":
    val_set = UrbanSoundDataset(configs, [10], transform=get_transform(configs))
val_loader = DataLoader(val_set, batch_size=24, shuffle=False,
                                num_workers=configs["DATALOADER"]["NUM_WORKERS"],
                                pin_memory=True, collate_fn = collate)

# Get the upper bound and lower bound on the values of the data, to be used as constraint in PGD
lower_bounds = []
upper_bounds = []
for step, (x, y, seq_lens) in enumerate(val_loader):    
    upper_bounds.append(torch.max(x))
    lower_bounds.append(torch.min(x))
lower_bound = min(lower_bounds)
upper_bound = max(upper_bounds)
print("Range of the input data is (%f, %f)" %(lower_bound, upper_bound))

path_to_checkpoint = os.path.join(project_dir, pretrained_path)
# Get the class weights, used in reloading the model
if configs['DATASET']['WEIGHT']=='NORMAL':
    weight = torch.tensor([28.9047, 14.8049,  4.5985,  2.4675,  4.4632, 19.5806]).to(device=device)
elif configs['DATASET']['WEIGHT']=='SQUARED':
    weight = torch.tensor([835.4845, 219.1843,  21.1461,   6.0885,  19.9205, 383.4014]).to(device=device)
else:
    weight = None

if configs["MODEL"]["CRNN"]["RANDOMISED_SMOOTHING"] == True:
    base_classifier = LitCRNN.load_from_checkpoint(path_to_checkpoint, cfg=configs, class_weights=weight, strict=False, map_location=device).eval()
    model = SmoothClassifier.load_from_checkpoint(checkpoint_path=path_to_checkpoint, cfg=configs, map_location=device, class_weights=weight, base_classifier = base_classifier.to(device=device)).eval()
else:    
    model = LitCRNN.load_from_checkpoint(path_to_checkpoint, cfg=configs, class_weights=weight, strict=False, map_location=device).eval()

fmodel = PyTorchModel(model, bounds=(lower_bound, upper_bound), device=device)

{'CHECKPOINT': {'SAVE_NAME': 'crnn-bmw-audi-rs-0.1', 'SAVE_PATH': '../weights/bmw/', 'SAVE_TOP_K': 1}, 'DATALOADER': {'BATCH_SIZE': 8, 'NUM_WORKERS': 20}, 'DATASET': {'ANNOTATION_PATH': '/nfs/students/winter-term-2020/project-1/datasets/BMW/meta/bmw.csv', 'AUGMENTATION': 'none', 'FOLDER_PATH': '/nfs/students/winter-term-2020/project-1/datasets/Brake_Noise', 'NAME': 'BMW', 'NOISE_PATH': '../datasets/MUSAN/free-sound/', 'VAL_FOLD': 1, 'WEIGHT': 'NORMAL'}, 'LOSS': 'cross_entropy', 'MODEL': {'CRNN': {'INCLUDE_TOP': True, 'INCLUDE_TRANSFORM': True, 'RANDOMISED_SMOOTHING': True}, 'NAME': 'LitCRNN', 'NUM_CLASSES': 6}, 'SOLVER': {'GAMMA': 0.5, 'LEARNING_RATE': 0.001, 'LOG_PATH': '../logs/bmw/', 'MAX_EPOCH': 125, 'MIN_EPOCH': 1, 'NUM_GPUS': 1, 'SIGMA': 0.1, 'STEP_SIZE': 25, 'WEIGHT_DECAY': 0.001}, 'TRANSFORM': {'HOP_LENGTH': 256}, 'ATTACK': True}
Data augmentation for BMW dataset: none
Range of the input data is (-1.000000, 0.999969)


In [5]:
# evaluate accuracy on clean data on a batch
it = iter(val_loader)
batch = next(it)
clips = batch[0].to(device)
labels = batch[1].to(device)
lengths = batch[2].to(device)   # used only for CRNN

# delete some variables to free up memory
del it
del val_loader
del val_set

# evaluate robustness with L-inf Fast Gradient Attack
torch.cuda.empty_cache()
attack = FGSM()

In [6]:
epsilons = 0.001
attack.run = types.MethodType(_run, attack)
attack.get_loss_fn = types.MethodType(_get_loss_fn, attack)
attack.value_and_grad = types.MethodType(_value_and_grad, attack)
raw, clipped, is_adv = attack(fmodel, clips, labels, epsilons=epsilons, original_lengths=lengths)

is_adv = is_adv.cpu()
clipped = clipped.cpu() #"Attacks samples"

forward is called
forward is called


In [7]:
out = model(clipped.to(device), lengths)
preds = torch.argmax(out, dim=1)
print("whether the attack is sucessful")
print(is_adv)
print("labels predicted using attack samples")
print(preds) # labels predicted using attack samples
print("labels predicted using original samples")
print(batch[1]) # labels predicted using original samples

forward is called
whether the attack is sucessul
tensor([ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True, False,  True,  True, False,
        False, False, False, False])
labels predicted using attack samples
tensor([3, 3, 3, 3, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 3, 3, 3, 3, 3, 3, 3])
labels predicted using original samples
tensor([0, 0, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3])
